In [6]:

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
import wikipedia
import os
from PyPDF2 import PdfReader

# 1. استخراج متن از صفحه ویکی‌پدیا
wikipedia.set_lang("fa")  # تنظیم زبان به فارسی
wiki_page = wikipedia.page("سیمرغ")  # نام صفحه ویکی‌پدیا
wiki_content = wiki_page.content  # استخراج متن

# تقسیم متن ویکی‌پدیا به chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
wiki_chunks = text_splitter.split_text(wiki_content)

# تبدیل chunks به Documents با متادیتا
wiki_documents = [
    Document(page_content=chunk, metadata={"source": "wikipedia"})
    for chunk in wiki_chunks
]

# 2. استخراج متن از فایل .txt
txt_file_path = "/simorgh.txt"  # مسیر فایل متنی
with open(txt_file_path, "r", encoding="utf-8") as file:
    txt_content = file.read()

# تقسیم متن .txt به chunks
txt_chunks = text_splitter.split_text(txt_content)

# تبدیل chunks به Documents با متادیتا
txt_documents = [
    Document(page_content=chunk, metadata={"source": "txt_file"})
    for chunk in txt_chunks
]

# 3. استخراج متن از فایل PDF
pdf_file_path = "/simorgh.pdf"  # مسیر فایل PDF
pdf_reader = PdfReader(pdf_file_path)
pdf_content = ""

# ترکیب متن تمام صفحات PDF
for page in pdf_reader.pages:
    pdf_content += page.extract_text()

# تقسیم متن PDF به chunks
pdf_chunks = text_splitter.split_text(pdf_content)

# تبدیل chunks به Documents با متادیتا
pdf_documents = [
    Document(page_content=chunk, metadata={"source": "pdf_file"})
    for chunk in pdf_chunks
]

# 4. ترکیب تمام Documents در یک لیست
all_documents = wiki_documents + txt_documents + pdf_documents

# چاپ تعداد کل اسناد
print(f"تعداد کل Documents ایجاد شده: {len(all_documents)}")


تعداد کل Documents ایجاد شده: 37


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:

from langchain.vectorstores import FAISS
from langchain_huggingface.embeddings import HuggingFaceEmbeddings
from langchain.docstore.document import Document

# بارگذاری مدل Embedding
model_name = "heydariAI/persian-embeddings"  # مدل مناسب برای زبان فارسی
hf_embedding = HuggingFaceEmbeddings(model_name=model_name)

# ایجاد FAISS VectorStore
vector_store = FAISS.from_documents(all_documents, embedding=hf_embedding)

# جستجوی سریع با تنظیم فیلتر
query = "سیمرغ در مسائل پزشکی چه نقشی دارد؟"
results = vector_store.similarity_search(
    query,
    k=2,  # تعداد نتایج
    filter={"source": "txt_file"}  # فیلتر کردن به اسناد متنی
)

# نمایش نتایج
for i, res in enumerate(results, 1):
    print(f"نتیجه {i}: {res.page_content} [منبع: {res.metadata['source']}]")


نتیجه 1: سیمرغ درمانگر نماد پزشکی ایران است که با الهام از اساطیر ایران باستان و مستندات شاهنامهٔ فردوسی طراحی شده‌است. این نشان آبان‌ماه ۱۳۹۲ رونمایی شد.

تاریخچه
سیمرغ درمانگر در آبان‌ماه ۱۳۹۲ با تصویب در شورایعالی سازمان نظام پزشکی، به عنوان نشان فرهنگی جامعهٔ صنفی پزشکی ایران، جایگزین نشان جام و مار شد. [منبع: txt_file]
نتیجه 2: نشان گرافیکی سیمرغ
نشان سیمرغ درمانگر که تمبر آن طی مراسمی در بهمن‌ماه ۱۳۹۲ رونمایی شد، توسط تورج صابری‌وند گرافیست و مدیر هنری دپارتمان متابرند (زیرمجموعهٔ گروه بین‌المللی توسعه پایدار) طراحی شده‌است.[۱][۳]

نشان جام و مار
مقالهٔ اصلی: جام هایجیا
در نشان جام و مار یونانی که تا به امروز در ایران و جهان به عنوان نماد جامعهٔ پزشکی شناخته شده‌است، مار تصویری از تندرستی را به ذهن متبادر می‌سازد و جام (ظرف) دارو، تنها نگاهی ابزاری به مقولهٔ درمان دارد. [منبع: txt_file]
